# Exercises
Data Acquisition

- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

- This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

1) Read the case, department, and source data into their own spark dataframes.

2) Let's see how writing to the local disk works in spark:
   - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
   - Inspect your folder structure. What do you notice?

3) Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


In [7]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import asc, desc

In [8]:
cases = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
cases.show(7)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [11]:
cases.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [9]:
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
dept.show(7)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 7 rows



In [12]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [10]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
source.show(7)

+---------+-------------------+
|source_id|    source_username|
+---------+-------------------+
|   100137|   Merlene Blodgett|
|   103582|        Carmen Cura|
|   106463|    Richard Sanchez|
|   119403|     Betty De Hoyos|
|   119555|     Socorro Quiara|
|   119868|Michelle San Miguel|
|   120752|     Eva T. Kleiber|
+---------+-------------------+
only showing top 7 rows



In [13]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



1) How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

2) How many Stray Animal cases are there?

In [15]:
cases.filter(cases.service_request_type == 'Stray Animal').count()

26760

3) How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [16]:
cases.filter(cases.dept_division == 'Field Operations').where(cases.service_request_type != "Officer Standby").count()

113902

4) Convert the council_district column to a string column.

In [17]:
cases = cases.withColumn("council_district", col("council_district").cast("string"))

In [18]:
cases.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



5) Extract the year from the case_closed_date column.

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [27]:
cases = cases.withColumnRenamed("SLA_due_date", "case_due_date")

In [28]:
cases.withColumn("case_closed", expr('case_closed == "YES"')).withColumn("case_late", expr("case_late == 'YES'"))

DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, case_due_date: string, case_late: boolean, num_days_late: double, case_closed: boolean, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: string]

In [29]:
cases.select("case_closed", "case_late").show(7)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|       NO|
|        YES|      YES|
|        YES|       NO|
|        YES|       NO|
+-----------+---------+
only showing top 7 rows



In [30]:
cases.select("case_opened_date", "case_closed_date", "case_due_date").show(7)

+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
|     1/1/18 6:28|    1/1/18 14:38| 1/31/18 8:30|
|     1/1/18 6:57|    1/2/18 15:32| 1/17/18 8:30|
+----------------+----------------+-------------+
only showing top 7 rows



In [31]:
fmt = "M/d/yy H:mm"
cases = (
    cases.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

cases.select("case_opened_date", "case_closed_date", "case_due_date").show(7)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
|2018-01-01 06:28:00|2018-01-01 14:38:00|2018-01-31 08:30:00|
|2018-01-01 06:57:00|2018-01-02 15:32:00|2018-01-17 08:30:00|
+-------------------+-------------------+-------------------+
only showing top 7 rows



In [53]:
cases = cases.withColumn("case_opened_date", regexp_extract("case_opened_date", r"\d+$", 0))
cases.select("case_opened_date").show(7)

+----------------+
|case_opened_date|
+----------------+
|                |
|                |
|                |
|                |
|                |
|                |
|                |
+----------------+
only showing top 7 rows



6) Convert num_days_late from days to hours in new columns num_hours_late.

In [58]:
cases = (
    cases.withColumn("num_hours_late", datediff(current_timestamp(), "case_due_date")*24)
)
    

cases.select("num_hours_late").show(7)

+--------------+
|num_hours_late|
+--------------+
|          5592|
|         29472|
|         29472|
|         29184|
|         29568|
|         28848|
|         29184|
+--------------+
only showing top 7 rows



7) Join the case data with the source and department data.

In [60]:
cases.join(dept, on=cases.dept_division== dept.dept_division).show(7)

+----------+----------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-------+--------------+----------------+--------------------+----------------------+-------------------+
|   case_id|case_opened_date|case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|zipcode|num_hours_late|   dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------+----------------+----------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-------+--------------+----------------+--------------------+----------------------+-------------------+
|1014127332|                |  

8) Are there any cases that do not have a request source?

In [80]:
cases.filter(cases.request_address == None).count()

0

9) What are the top 10 service request types in terms of number of requests?

In [89]:
cases.sort(cases.service_request_type).show(7)

+----------+----------------+----------------+-------------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+------------+--------------------+----------------+-------+--------------+
|   case_id|case_opened_date|case_closed_date|      case_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|   source_id|     request_address|council_district|zipcode|num_hours_late|
+----------+----------------+----------------+-------------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+------------+--------------------+----------------+-------+--------------+
|1013991533|                |             +\d|2018-01-16 08:30:00|       NO| -41.75542824|        YES|Code Enforcement|"Vacant Lot-City ...|77.01597222|     Closed|CRM_Listener|6400  MARCUM, San...|               6|  78227|         29208|
|1013994713|                |             +\

In [105]:
cases.groupBy("service_request_type").count().orderBy('count', ascending=False).show(10, truncate = False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



10) What are the top 10 service request types in terms of average days late?

11) Does number of days late depend on department?

12) How do number of days late depend on department and request type?